In [10]:
import chess.pgn
import numpy as np
import os, glob
import shlex
import chess
import chess.pgn
import unicodedata
import os

Here for subvariations in parenthesis

https://tex.stackexchange.com/questions/5727/typesetting-nested-chess-variations-in-skak

como:

 {\variation[invar]{1... Qxg3 2. Nf6+ } (\variation[invar]{2. Nxg3 gxh6})(\variation[invar]{ 2... Rxf2 3. Qh5})(\variation[invar]{2... Rf6 3. Qg5})}    

In [32]:



#use these tags only
tags = [
    "Event",
    "Site",
    "Date",
    "Round",
    "White",
    "Black",
    #"Opening",
    #"ECO",
    "Result"
]


results = {
    "1-0":"\\whiteWins",
    "0-1":"\\blackWins",
    "1/2-1/2":"\\aDraw",
    "*":""
        }

results2 = {
    "1-0":"\\rwwins",
    "0-1":"\\rbwins",
    "1/2-1/2":"\\rdraw",
    "*":""
        }
#we do "not" remove \ or {}, because the user should be allowed to enter latex commands in pgn
specialLatex = {
    "#":"\\#",
    "&":"\\&",
    "%":"\\%",
    "$":"\\$",
    "_":"\\_",
    "^":"\\^",
    }

glyphs = [
    "",
    "!",
    "?",
    "!!",
    "??",
    "!?",
    "?!",
    "#"
    ]

replace = {
#merge comments
"}{":" ",
        }

#Latex line break
BR = "\n"
PAR = "\\par "

numbers = list(range(10))
numbers = list(map(str,numbers))

nags = list(range(1,256))
nags = list(map(str,nags))
nags = list("$"+val for val in nags)
nags = nags[::-1]

num2alph = {
        " ":"",
        "1":"a",
        "2":"b",
        "3":"c",
        "4":"d",
        "5":"e",
        "6":"f",
        "7":"g",
        "8":"h",
        "9":"i",
        }

#for handling special characters

#no chess comment
miscSymbols = {
176:"{°}", 
196:"{\\\"A}", 
214:"{\\\"O}",
215:"{\\ensuremath{\\times}}", 
220:"{\\\"U}",
223:"{\\ss}",
228:"{\\\"a}",
246:"{\\\"o}",
252:"{\\\"u}",
710:"{\\^}", 
8201:"{\\,}", 
8208:"{-}", 
8209:"{\\-/}", 
8211:"{--}", 
8212:"{---}", 
8213:"{---}", 
8216:"{`}", 
8217:"{'}", 
#german only
#8218:"{\\ensuremath{\\glq}}", 
8220:"{``}", 
8221:"{''}", 
#8222:"{\\ensuremath{\\glqq}}",
8491:"{\\AA}",
8364:"{}",
8764:"{\\ensuremath{\\sim}}", 
9812:"{\\symking}",
9813:"{\\symqueen}",
9814:"{\\symrook}",
9815:"{\\symbishop}",
9816:"{\\symknight}",
9817:"{\\sympawn}",
9818:"{\\symking}",
9819:"{\\symqueen}",
9820:"{\\symrook}",
9821:"{\\symbishop}",
9822:"{\\symknight}",
9823:"{\\sympawn}",
64257:"", 
64258:"", 
}

#chess
chessSymbols = {
33:"!",
35:"#",
43:"+",
45:"-",
61:"=",
63:"?",
171:"{\\qside}",
177:"{\\wdecisive}",
187:"{\\kside}",
8252:"{!!}",
8263:"{??}",
8265:"{!?}",
8264:"{?!}",
#zero length non-breaking space
8288:"",
8593:"{\\withattack}",
8594:"{\\withinit}",
8646:"{\\counterplay}",
8711:"\\ensuremath{\\nabla}", 
8723:"{\\bdecisive}",
8734:"{\\unclear}", 
8644:"{\\counterplay}",
8660:"{\\file}",
8663:"{\\diagonal}",
8804:"{\\ensuremath{\\leq}}", 
8805:"{\\ensuremath{\\geq}}",
8853:"{\\timelimit}",
8862:"{\\centre}",
8869:"{\\ending}",
8722:"{--}",
8979:"{\\betteris}",
9632:"{\\onlymove}",
9633:"{\\onlymove}",
9651:"{\\withidea}",
9672:"{\\moreroom}",
10227:"{\\devadvantage}",
10752:"{\\zugzwang}",
10866:"{\\wbetter}",
10865:"{\\wupperhand}",
58176:"{\\bishoppair}",
58177:"{\\opposbishops}",
58178:"{\\samebishops}",
58179:"{\\passedpawn}",
58180:"{\\compensation}",
}

#replace x (like in 1. e4 d5 2. exd) with proper capture symbol
def replace_x(s):
    if s == "":
        return ""
    out = s[0]
    for charN in range(1,len(s)-1):
        if s[charN] == "x":
            if any(s[charN-1] == str(number) for number in range(1,10) ) and s[charN+1] in [chr(i) for i in range(ord('a'),ord('h')+1)]:
                out += "{\\ensuremath{\\times}}"
                continue
        out += s[charN]
    return out + s[-1]

combinedSymbols = {
        "\\mbox{+{--}}":"{\\wdecisive}",
        "\\mbox{{--}+}":"{\\bdecisive}",
        }

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
#
#the uni2tex function is based on an answer on stackexchange by Khaled Hosny in 2011:
#https://tex.stackexchange.com/questions/23410/how-to-convert-characters-to-latex-code
#
accents = {
    0x0300: '`', 0x0301: "'", 0x0302: '^', 0x0308: '"',
    0x030B: 'H', 0x0303: '~', 0x0327: 'c', 0x0328: 'k',
    0x0304: '=', 0x0331: 'b', 0x0307: '.', 0x0323: 'd',
    0x030A: 'r', 0x0306: 'u', 0x030C: 'v', 
}

def uni2tex(text, gameNr):
    out = ""
    txt = tuple(text)
    #search non-breaking empty spaces and replace them with mboxes
    while text.find(chr(8288)) != -1:
        pos = text.find(chr(8288))
        text = text[:pos-1]+"\\mbox{"+text[pos-1]+text[pos+1]+"}"+text[pos+2:]
    for i in range(len(text)):
        char = text[i]
        code = ord(char)
        if unicodedata.category(char) in ("Mn", "Mc") and code in accents:
            out += "\\%s{%s}" %(accents[code], txt[i+1])
            i += 1
        #precomposed characters
        elif len(unicodedata.decomposition(char).split()) == 2 and is_number(unicodedata.decomposition(char).split()[0]):
            base, acc = unicodedata.decomposition(char).split()
            acc = int(acc, 16)
            base = int(base, 16)
            if acc in accents:
                out += "\\%s{%s}" %(accents[acc], chr(base))
            else:
                out += char
        elif code > 127:
            if code in chessSymbols:
                out += chessSymbols[code]
            elif code in miscSymbols:
                out += miscSymbols[code]
            #if an improper (not combine) macron is used after a number
            elif code == 175:
                before = text[i-1]
                out = out [:i-2]
                out += "\\={" + before + "}"
            else:
                if i > 20:
                    print("unrecognized symbol in game",gameNr, ":", char, "position:", i+1, "char code:", code,  "\"",text[i-20 : i+1], "\"")
                else:
                    print("unrecognized symbol in game",gameNr, ":", char, "position:", i+1, "char code:", code,  "\"",text[ : i+1], "\"")
        else:
            out += char
        # combining marks
    for sym in combinedSymbols:
        out = out.replace(sym,combinedSymbols[sym])
    return out


def processGame(content,write,fenboard):
    if content == "" or content in results.keys():
        print("empty game")
        write.write("\\end{samepage}"+"\n\n")
        return
    #Aquarium diagram markers to Chessbase markers for simplicity
    diagramAqua = ["[%t Dgrm] DA", "[%t bDgr] DB"]
    for aquaMarker in diagramAqua:
        content = content.replace(aquaMarker,"Diagramm #")
    
    #remove special comments if the whole comment is "special"
    while content.find("[%") != -1:
        start = content.find("[%")
        subst = ""
        aVar = 0
        while content[start+aVar] != "]":
            subst += content[start+aVar]
            aVar += 1
        subst+="]"
        content = content.replace(subst,"")
    #remove empty comments
    content = content.replace("{}","%%%%%%%%%")
    content = content.replace("{ }","%%%%%%%%%")
    
    #NAGs translation
    #these NAGs occur only at the beginning of variations and need to be in front of the move.
    specialNAG = ["$140","$141","$142","$143","$144","$145"]
    for nag in specialNAG:           
        while content.find(nag) != -1:
            start = content.find(nag)
            bVar = 0
            spaces = 0
            #there are at least 2 spaces before we are in front of the move before
            spacesMax = 2
            while spaces < spacesMax:
                #if a variation starts, we are right as well
                if content[start-bVar] == "(":
                    spaces += 1
                elif content[start-bVar] == " ":
                    spaces += 1
                    #put NAG in front of move number
                    if content[start-bVar-1] == ".":
                        spacesMax += 1
                    #skip another NAG
                    if content[start-bVar+1] == "$" and bVar > 1:
                        spacesMax += 1
                bVar += 1
            content = content[:start-bVar+2]+"%%%"+nag[1:]+"%%%"\
            +content[start-bVar+2:start]+content[start+4:]

    #add missing move numbers for the first black move after a comment
    index = 1
    while index < len(content):
        index = content.find("}", index)
        if index == -1:
            break
        #} 1... e5 is the correct syntax
        index += 1
        #ok, space is not interesting
        while content[index] == " ":
            index += 1
        #ignore our NAG comments  
        if content[index] == "%":
            index += 9
        #ok, space is not interesting
        while content[index] == " ":
            index += 1
        if index >= len(content):
            break
        #no move after }?
        ignoreThese = [")","(","}","{","*"]
        if content[index] in ignoreThese:
            continue
        #black moves should start with N..., but sometimes that is forgotten
        if content[index] not in numbers:
            #ok, we need to do something!
            index2 = index
            while content[index2] != "." or content[index2-1] not in numbers:
                
                #skip comments
                if content[index2] == "}":
                    bracount = 1
                    index2 -= 1
                    while content[index2] != "{" or bracount != 0:
                        index2 -= 1
                        if content[index2] == "{":
                            bracount -= 1
                        if content[index2] == "}":
                            bracount += 1
                #skip variations
                elif content[index2] == ")":
                    bracount = 1
                    index2 -= 1
                    while content[index2] != "(" or bracount != 0:    
                        index2 -= 1    
                        
                        #skip comments
                        if content[index2] == "}":
                            bracount2 = 1
                            index2 -= 1
                            while content[index2] != "{" or bracount2 != 0:
                                index2 -= 1
                                if content[index2] == "{":
                                    bracount2 -= 1
                                if content[index2] == "}":
                                    bracount2 += 1
                    
                        if content[index2] == "(":
                            bracount -= 1
                        if content[index2] == ")":
                            bracount += 1
                index2 -= 1
            #ok, search for number was successfull 
            index2 -= 1
            index3 = index2-1
            #ok, index2 is the start of the number, get the whole number
            while content[index3] in numbers:
                index3 -= 1

            number = str(int(content[index3+1:index2+1]))
            content = content[:index]+number+"... "+content[index:]
            index += len(number)+5
        
    #remove double spaces
    while (content.find("  ") != -1):
        content = content.replace("  "," ")           

    #remove wrong N... moves, ignore the first move of the game
    index = 6
    while index < len(content):
        index = content.find("...", index)
        if index == -1:
            break
        index2 = index-1
        #ok, now find the position where the move number ends
        while (content[index2] != " "):
            index2 -= 1
            #skip our NAG markers
            if content[index2] == " " and content[index2-1] == "%":
                index2 -= 1
        #make sure it is a number
        if content[index2+1:index].isdigit():
            if content[index2-1] != "{" and content[index2-1] != "}" and content[index2-2] != "}" and content[index2-2] != "{"\
            and content[index2-1] != "(" and content[index2-1] != ")" and content[index2-2] != ")" and content[index2-2] != "(":
                content = content[:index2+1]+content[index+3:]
        index += 3
    #remove double spaces
    while (content.find("  ") != -1):
        content = content.replace("  "," ")        

    #detemine structure
    contentNoSpace = content.replace(" ","")
    level = 0
    structure = []
    comment = 0
    for i in range(len(contentNoSpace)):
        if contentNoSpace[i] == "{":
            comment = 1
        elif contentNoSpace[i] == "}":
            comment = 0
        elif contentNoSpace[i] == "(" and comment == 0:
            level+=1
            structure.append([level,0])
        elif contentNoSpace[i] == ")" and comment == 0:
            level-=1
            #check if another line follows directly
            if contentNoSpace[i+1] != "(":
                structure.append([level,1])
            else:
                structure.append([-1,1])
    
    #count how many entries per level
    levels = [0] * 10
    #how many entries per level occurence
    levelsN = [[] for i in range(10)]
    for i in range(len(structure)):
        if structure[i][0] != 0 and structure[i][0] != -1:
            level = structure[i][0]
            if level > 9:
                print("too many variation levels, stopping!")
                raise SystemExit
            if structure[i][1] == 0:    
                levels[level] += 1
            if structure[i+1][0] == (level - 1):
                levelsN[level].append(levels[level])
                levels[level] = 0
#    print(levelsN)                    
#    print("structure:",structure)
    symbols = [[] for i in range(10)]
    for i in range(10):
        if len(levelsN[i]) != 0:
            for j in range(len(levelsN[i])):
                if levelsN[i][j] > 1:
                    for k in range(levelsN[i][j]):
                        symbols[i].append(str(k+1))
                else:
                    symbols[i].append("")
#    print(symbols)   
    brackets = []
    closedBracketsN = 0
    level = 0
    levels = [0] * 10
    symbol = ""
    comment = 0
    printDiagram = ""
    for i in range(len(content)):
        if content[i] == "{":
            comment = 1
        if content[i] == "}":
            comment = 0
        if content[i] == "(" and comment == 0:
            level += 1
            symbol += symbols[level][levels[level]]
            if symbols[level][levels[level]] == "" and level > 1:
                symbol += " "
#            print(symbol)
            levels[level] += 1
            if len(symbol) > 1:
                if symbol.find(" ") != -1:
                    brackets.append("")
                else:
                    brackets.append(num2alph[symbol[0]]+symbol[1:])
            elif len(symbol) == 1:
                brackets.append(num2alph[symbol[0]])
            else:
                brackets.append(symbol)
        if content[i] == ")" and comment == 0:
            level-=1
            symbol = symbol[:-1]
            closedBracketsN += 1
    
    #print(symbols)
    if fenboard != "":
        output = "\\mainline[level=1]{"
    else:
        output = "\\firstmainline[level=1]{"
    level = 0
    #how many brackets open
    bracket = 0
    closedBrackets = 0
    #open brackets
    bracketOpen = []
    #inside a comment?
    comment = 0
    text = ""
    #separate introduction for variation
    pretext = ""
            
    iterall = iter(range(len(content)))
    for i in iterall: 
        if content[i] == "{" and comment == 0:
            comment = 1
            if level == 0:
                pretext += "}\n "
            else:
                pretext += "\\xskakcomment{ "
        elif content[i] == "}" and comment == 1:
            comment = 0
            #replace # with \#
            diagramMark = [
                    #Chessbase - some stupid guesses :-)
                    "Diagramm #",
                    "Diagram #",
                    "Diagrama #",
                    "Диаграма #",
                    "Dijagram #",
                    "Skeem #",
                    "Kaavio #",
                    "Διάγραμμα #",
                    "Skýringarmynd #",
                    "Léaráid  #",
                    "Diagramma  #",
                    "Диаграм #",
                    "Dijagramma #",
                    "Схема #",
                    #an internal Marker
                    "DiaW#",
                    "DiaB#",
                    "#DiaW",
                    "#DiaB"
                    ]        
            #Diagram from blacks point of view (inverse)?
            diagramMarkBlack = [
                    #"DiaB#",
                    #"#DiaB"
                    ]
            for marker in diagramMark:
                if text.find(marker) != -1:
                    #Diagram from blacks point of view (inverse)?
                    inverseDiagram = False
                    if marker in diagramMarkBlack:
                        inverseDiagram = True
                    text = text.replace(marker,"")
                    #main line is simple
                    if level == 0:
                        if inverseDiagram:
                            pretext = "\\dia}\n{\\par \\centering\\chessboardn[inverse] \\par}\n{"+pretext
                        else:
                            pretext = "\\dia}\n{\\par \\centering\\chessboardn \\par}\n{"+pretext
                    #now the fun starts - a diagram inside a variation :-/
                    else:
                        #parse the moves before
                        getmoves = ""
                        cVar = 0
                        #ignore comments and lines
                        while cVar <= i:
                            bracount = 0
                            #skip comments
                            if content[i-cVar] == "}":
                                bracount = 1
                                cVar += 1
                                while content[i-cVar] != "{" or bracount != 0:
                                    cVar += 1
                                    if content[i-cVar] == "{":
                                        bracount -= 1
                                    if content[i-cVar] == "}":
                                        bracount += 1
                            #skip variations
                            elif content[i-cVar] == ")":
                                bracount = 1
                                cVar += 1
                                while content[i-cVar] != "(" or bracount != 0:    
                                    cVar += 1    
                                    
                                    #skip comments
                                    if content[i-cVar] == "}":
                                        bracount2 = 1
                                        cVar += 1
                                        while content[i-cVar] != "{" or bracount2 != 0:
                                            cVar += 1
                                            if content[i-cVar] == "{":
                                                bracount2 -= 1
                                            if content[i-cVar] == "}":
                                                bracount2 += 1
                                
                                    if content[i-cVar] == "(":
                                        bracount -= 1
                                    if content[i-cVar] == ")":
                                        bracount += 1
                            else:
                                getmoves += content[i-cVar]
                            cVar += 1
                            
                        getmoves = getmoves[::-1]
                        for nagType in nags:
                            getmoves = getmoves.replace(nagType,"")
                        getmoves = getmoves.replace("( ","(")
                        #make sure a "(" has a space in front
                        getmoves = getmoves.replace("("," (")
                        while getmoves.find("  ") != -1:
                            getmoves = getmoves.replace("  "," ")
                            
                        #remove the move in front of a bracket (variation)
                        while getmoves.find("(") != -1:
                            start = getmoves.find("(")
                            spaces = 0
                            cVar = 0
                            while spaces < 2:
                                cVar += 1
                                if getmoves[start-cVar] == " ":
                                    spaces += 1
                            if getmoves[start-cVar-1] == ".":
                                cVar += 1
                                while getmoves[start-cVar] != " " and start > cVar:
                                    cVar += 1
                            getmoves = getmoves[:start-cVar+1]+getmoves[start+1:]
                            
                        #now remove N... moves
                        while getmoves.find("...") != -1:
                            start = getmoves.find("...")
                            cVar = 1
                            while getmoves[start-cVar] in numbers:
                                cVar += 1
                            getmoves = getmoves[:start-cVar+1]+getmoves[start+4:]
                        #delete leading spaces
                        while getmoves[0] == " ":
                            getmoves = getmoves[1:]
                        #moves browsed!
                        #create a pgn
                        if fenboard != "":
                            getmoves = "[SetUp \"1\"]\n[FEN \""+fenboard+"\"]\n\n"+getmoves
                        else:
                            getmoves = "\n"+getmoves
                        getmoves = "[Event \"?\"]\n[Site \"?\"]\n[Date \"????.??.??\"]\
                        \n[Round \"?\"]\n[White \"Test\"]\n[Black \"Test\"]\
                        \n[Opening \"?\"]\n[ECO \"?\"]\n[Result \"1-0\"]\n"+getmoves
                        #get FEN
                        try:
                            from StringIO import StringIO  # Python 2
                        except ImportError:
                            from io import StringIO  # Python 3
                        pgn = StringIO(getmoves)
                        #print(getmoves)
                        game = chess.pgn.read_game(pgn)
                        fen = game.end().board().fen()
                        pos = pretext.find("\\xskakcomment{")
                        pretext = pretext[:pos+14] + "\\diav{}" + pretext[pos+14:]
                        if inverseDiagram:
                            printDiagram += "\n{\\par \\centering\\chessboardt[inverse,setfen="+fen+"] \\par}\n"
                        else:
                            printDiagram += "\n{\\par \\centering\\chessboardt[setfen="+fen+"] \\par}\n"
            #ignore colour markers from Aquarium!
            if text.find("[%t Ctrl]") == -1:
#                print("Warning! Skipping comment to coloured move!")
            #remove time markers and so on
#            else:
                while text.find("[%") != -1:
                    start = text.find("[%")
                    subst = ""
                    aVar = 0
                    while text[start+aVar] != "]":
                        subst += text[start+aVar]
                        aVar += 1
                    subst+="]"
                    text = text.replace(subst,"")
                #if not empty now, add comment
                if text.replace(" ","") != "" or pretext.find("\\dia") != -1:
                    if level == 0:
                        #check if special chess symbols after the move,
                        #which should be placed directly after it
                        shortComment = False
                        stopShort = False
                        newText = ""
                        for char in text:
                            if ord(char) not in chessSymbols:
                                stopShort = True
                            if stopShort:
                                newText += char
                            else:
                                if shortComment == False:
                                    shortComment = True
                                    output += " \\xskakcomment{"
                                output += char
                        if shortComment:
                            output += "}"
                        text = newText
                        #check if comment is empty
                        if text.replace(" ","") != "" or pretext.find("\\chessboard") != -1:
                            text = replace_x(text)
                            text += "\n\n\\mainline[level=1]{"
                        else:
                            text = ""
                            pretext = ""
                    else:
                        #put chess symbols directly after move
                        shortComment = False
                        stopShort = False
                        newText = ""
                        newMoveText = ""
                        for char in text:
                            if ord(char) not in chessSymbols:
                                stopShort = True
                            if stopShort:
                                newText += char
                            else:
                                if shortComment == False:
                                    shortComment = True
                                newMoveText += char
                        pos = pretext.find("\\xskakcomment{")
                        pretext = pretext[:pos+14] + newMoveText + pretext[pos+14:]
                        text = newText
                        #remove space if empty comment
                        if text.replace(" ","") == "":
                            if pretext[-1] == " ":
                                pretext = pretext[:-1]
                                #mark the short comment
                                pretext += "}***"
                        else:
                            text = replace_x(text)
                            text += "}} \\variation[level="+str(level+1)+"]{"
                    for key in specialLatex.keys():
                        text = text.replace(key,specialLatex[key])
                    output += pretext+text
            text = ""
            pretext = ""
            
        elif content[i] == "(" and comment == 0:
            
            bracketOpen.append(1)
            level += 1
            output += "}"
            output += printDiagram
            printDiagram = ""
            #no line finished or started before
            if content[i-2:i].find(")") == -1 and content[i-2:i].find("(") == -1:
                #no enumeration
                if brackets[bracket] == "":
                    #new line if comment before
                    if content[i-1] == "}" or content[i-2] == "}":
                        if level == 1:
                            output += BR
                    if level > 1:
                        output += " ("
#                    else:
#                        output += "\\noindent{}"
            
            if level == 1:
                output += BR+BR
            bufout = "\\variation[level="+str(level+1)+"]{"
            
            #if enumeration
            if brackets[bracket] != "":
                spaces = ""
                if content[i-1] == "}" or content[i-2] == "}":
                    #if first entry of enumeration after comment in the main line
                    if level == 1 and brackets[bracket][0] == "a":
                        bufout = BR+BR+bufout
                if brackets[bracket][-1] != "a":
                    if level == 1:
                        bufout = BR+BR+bufout
                if level > 1:
                    bufout = "\\\\"+bufout
#                if (len(brackets[bracket])-1) > 0:
#                    spaces += "\\hspace*{"+str(.5*(len(brackets[bracket])-1))+"mm}"
                bufout += "\\xskakcomment{\\noindent\\textbf{"+brackets[bracket]+")} } "
                            
            output += bufout
	    
            bracket += 1

        elif content[i] == ")" and comment == 0:
            closedBra = 0
            closedBrackets += 1
            for bra in range(len(bracketOpen)):
                if bracketOpen[-1-bra] == 1:
                    bracketOpen[-1-bra] = 0
                    closedBra = len(bracketOpen)-1-bra
                    break
            level -= 1
            output += "}"
            
            #no enumeration?
            if brackets[closedBra] == "":
                if level > 0:
                    output += ")"
                #check if several brackets close:
                if content[i+1:i+3].find(")") == -1:
                    output += printDiagram
                    printDiagram = ""
            else:
                #check if several brackets close:
                if content[i+1:i+3].find(")") == -1:
                    output += printDiagram
                    printDiagram = ""
                if level > 0:
                    output += " "
            #last bracket, prevent page break for result - seems not to work
#            if closedBrackets == closedBracketsN:
#                output += "\\begin{samepage}"
            if level == 0:
                output += "\n\n\\mainline[level=1]{"
            else:
                output += "\n\\variation[level="+str(level+1)+"]{"
        elif comment == 1:
            text += content[i]
        elif content[i] == "%":
                #we marked the NAGs that are in front of moves and replace them now:
                if content[i:i+3] == "%%%":
                    if content[i:i+9] == "%%%140%%%":
                        output += "\\xskakcomment{\\withidea} "
                    if content[i:i+9] == "%%%141%%%":
                        output += "\\xskakcomment{$\\nabla$} "
                    if content[i:i+9] == "%%%142%%%":
                        output += "\\xskakcomment{\\betteris} "
                    elif content[i:i+9] == "%%%143%%%":
                        output += "\\xskakcomment{\\worseis} "
                    elif content[i:i+9] == "%%%144%%%":
                        output += "\\xskakcomment{\\equalis} "
                    elif content[i:i+9] == "%%%145%%%":
                        output += "\\xskakcomment{\\chesscomment} "
                    elif content[i:i+9] == "%%%%%%%%%":
                        output += "{}"
                    for dummy in range(8):
                        next(iterall)
                else:
                    output += content[i]
        elif content[i] == "-":
            #ignore non-nullmove minus, main line null moves and null moves in front of comments
            if content[i:i+2] == "--" and level > 0 and content[i+3:i+5].find("{") == -1:
                 #white move in front
                if content[i-2] == "." and content[i-3] != ".":
                    output += "\\xskakcomment{. -- }"
                    spacesMax = 2
                    spaces = 0
                    start = i+1
                    aVar = 1
                    while spaces < spacesMax:
                        if content[start+aVar] == " ":
                            spaces += 1
                            if content[start+aVar+1] == "$":
                                spacesMax += 1
                        if content[start+aVar] == ")":
                            break
                        if spaces != 1 or content[start+aVar] != " ":
                            output += content[start+aVar]
                        next(iterall)
                        aVar += 1
                    output += "} \\variation[level="+str(level+1)+"]{"     
                else:
                    output += "}\\variation[level="+str(level+1)+"]{\\xskakcomment{ -- }"
                next(iterall)
            else:
                output += content[i]
        else:
            output += content[i]
    noResult = False
    if output[-2:] == "* ":
        output = output[:-2]+"}"
        noResult = True
    elif output[-4:] == "1-0 ":
        output = output[:-4]+"} "+results2["1-0"]
    elif output[-4:] == "0-1 ":
        output = output[:-4]+"} "+results2["0-1"]
    elif output[-8:] == "1/2-1/2 ":
        output = output[:-8]+"} "+results2["1/2-1/2"]
           
    #clean up output - some "dirty" tricks to handle rare cases neglected above and simplifying the code
    while (output.find("  ") != -1):
        output = output.replace("  "," ")
    output = output.replace("\n\\mainline[level=1]{ }","")
    output = output.replace("\n\\mainline[level=1]{}","")
    output = output.replace("\\mainline[level=1]{ }","")
    output = output.replace("\\mainline[level=1]{}","")
    for i in range(10):
        output = output.replace("\n\\variation[level="+str(i+1)+"]{ }","")
        output = output.replace("\n\\variation[level="+str(i+1)+"]{}","")
        output = output.replace("\\variation[level="+str(i+1)+"]{ }","")
        output = output.replace("\\variation[level="+str(i+1)+"]{}","")
    output = output.replace(" \\dia","\\dia")
    #space before enumeration
#    output = output.replace("\\variation[level=2]{\\xskakcomment{\\noindent\\textbf{" + PAR, PAR + "\\variation[level=2]{\\xskakcomment{\\noindent\\textbf{")
    
    #empty line
#    output = output.replace("\\noindent \n","")
    #empty {}
    output = output.replace("\n{}\n","\n")
    #many empty lines
    output = output.replace("\n \n","\n")
    for i in range(10):
        output = output.replace("\n\n\n","\n\n")
    #bracket closing variation in empty line
    output = output.replace("\\\\ )",")")
    #remove empty game body
    output = output.replace("\\mainline[level=1]{ 1. }","")
    #no space after diagram
#    output = output.replace("\\end{center}\n\n\\noindent"+PAR,"\\end{center}\n\\noindent ")
    output = output.replace("} )","})")
    output = output.replace("1/2-1/2","\\aDraw")
    output = output.replace("1-0","\\whiteWins")      
    output = output.replace("0-1","\\blackWins")
    #comment at the begin of a variation: remove leading space
    output = output.replace("]{\\xskakcomment{ ","]{\\xskakcomment{")
    #but not for null move
    output = output.replace("]{\\xskakcomment{--","]{\\xskakcomment{ --")
    #remove space after null move followed by new variation
    output = output.replace("-- } } (\\variation[level=","--} } (\\variation[level=")
    #remove empty comments:
    output = output.replace("\\xskakcomment{ }","")
    output = output.replace("\\xskakcomment{}","")
    #comments starting with a colon
    output = output.replace("\\xskakcomment{ ,","\\xskakcomment{,")
    #Diagram and enumeration right away
#    output = output.replace("\\end{center}\n\n\\noindent \\variation[level=2]{\\xskakcomment{\\textbf{a","\\end{center}\n\n\\noindent \\variation[level=2]{\\xskakcomment{\\textbf{a")
    #no spaces in front of enumeration first entry
#    output = output.replace("}\\variation[level=2]{\\xskakcomment{\\noindent\\textbf{"+VSPACE+"a","}\\variation[level=2]{\\xskakcomment{\\noindent\\textbf{a")
#    output = output.replace("} \\textbf{)}","}\\textbf{)}")
    #removing X... move numbers caused by removing short comments
    index = 1
    while index < len(output):
        index = output.find("...", index)
        if index == -1:
            break
        index2 = index-1
        #ok, now find position when move number ends
        while (output[index2] != " "):
            index2 -= 1
        if output[index2+1:index].isdigit():
            if output[index2-1] != "{" and output[index2-1] != "}" and output[index2-2] != "}" and output[index2-2] != "{":
                output = output[:index2+1]+output[index+3:]
            elif output[index2-4:index2-1] == "***":
                output = output[:index2+1]+output[index+3:]
            #check if xskakcomment in front
            elif output[index2-1] == "}" or output[index2-2] == "}":
                indexN0 = index2-1
                while output[indexN0] != "{":
                    indexN0 -= 1
                indexN1 = indexN0-1
                while output[indexN1] != "\\":
                    indexN1 -= 1
                if output[indexN1+1:indexN0] == "xskakcomment":
                    if output[indexN1-1] != "{" and output[indexN1-2] != "{" or output[indexN1-4:indexN1-1] == "***":
                        output = output[:index2+1]+output[index+3:]
                
        index += 3
    #remove short comment markers
    output = output.replace("***","")
    #removing special chars
    output = uni2tex(output,gameNo)
    #make sure the result is on the same page as the last main line
    if not noResult:
        position = output.rfind("\\mainline")
        if output.rfind("\\variation") > position:
            position = output.rfind("\\variation")
        if output.rfind("\\noindent") > position:
            position = output.rfind("\\noindent")
        output = output[:position]
        #+"\\begin{samepage}"+output[position:]
        #output += "\\end{samepage}"
    
    while (output.find("  ") != -1):
        output = output.replace("  "," ")
    write.write(output+"\n\n")
    print("Game processeda.")
    return output
    


In [33]:
nf = 'Liquidation'
foldern = f'{nf}/'
listofpgn = glob.glob(f"{foldern}/*.pgn")
#pgn = open('blundersmapicharwhite0.pgn')


In [34]:
listofpgn

['Liquidation/nopushLiquidationBenjamin.pgn']

In [35]:
n = f'{foldern}myblunders.tex'

with open(n,'w') as f:


    f.write('\\documentclass{book}\n')
    f.write('\\usepackage[ps,mover]{skak}\n')
    f.write('\\usepackage{xskak}\n')
    f.write('\\begin{document}\n')
    f.write('\\parindent=0pt \n')
    f.write('\\tableofcontents\n')



    for nugame,gamepgn in enumerate(listofpgn):
        
        f.write(f'\chapter{{{nugame}}}\n')
        dividedgames = []

        gamechap = []
        pgn = open(gamepgn)



        turnb = 'Black to move'
        turnw = 'White to move'
        turn = turnw



        while True:
            game = pgn.tell()
            gamepgn = chess.pgn.read_game(pgn)
            #print(headers.get('White'))
            if gamepgn is None:
                break
            #if gamepgn.headers['White'] == turn:
            gamechap.append(gamepgn)
            dividedgames.append(gamechap)



        for gameNo,g in enumerate(gamechap):
            try:
                game = g#gamechap[0]

                #game = g#gamechap[0]

                if 'w' in game.headers['FEN']:
                    turntoplay = True
                else: 
                    turntoplay = False
                    
                mainlinestemp = str(game.mainline_moves())
                fentemp = game.headers['FEN']
            except: 
                pass


            
            #Title of board
            #titleboard = str(game.headers['White'],
            f.write(f"\\section{{{game.headers['White']}}}\n")
            f.write(f"\\subsection{{{game.headers['Black']}}}\n")

            f.write(f"\\fenboard{{{fentemp}}}\n")
            f.write('\\begin{center}\n')
            #f.write(f"\\showboard \n")
                #\chessboard[{}},inverse=false]
            if turntoplay:
                f.write(f"\\showboard \n")
            else:
                f.write(f"\\showinverseboard \n")

            #f.write(f"\\chessboard[setfen={fentemp},inverse={turntoplay}]\n")
            f.write('\\end{center}\n')

            #f.write(f"\\mainline{{{mainlinestemp}}}")
            f.write(f"\\clearpage \n")
            f.write(f"\\newpage \n")


            f.write(f"\\mainline{{{mainlinestemp}}}")

            f.write(f"\\fenboard{{{fentemp}}}\n")
            f.write('\\begin{center}\n')

            if turntoplay:
                f.write(f"\\showboard \n")
            else:
                f.write(f"\\showinverseboard \n")


            #f.write(f"\\chessboard[setfen={fentemp},inverse={inv}]\n")
            f.write('\\end{center}\n')

            f.write(f"\\fenboard{{{fentemp}}}\n")

            #if nugame == len(listofpgn)-1:
            #    f.write("\\end{document}\n")




            #Write all
            c1 = str(g).split(']')[-1].strip()
            t3 =  processGame(c1,f,fentemp)
            #print(c1)
            #print(t3)
            print(fentemp)


            f.write('\n')
            f.write(f"\\clearpage \n")
            f.write(f"\\newpage \n")
            f.write('\n')

    f.write('\\end{document}\n')






Game processeda.
8/p7/4P3/1k3p2/3Q1P1p/5K1P/6P1/4q3 w - - 0 84
Game processeda.
8/3q1p2/4p1kp/1PQ5/3P2p1/6P1/5PKP/8 w - - 0 35
Game processeda.
8/2p3pk/7p/2Q2p2/5P2/3q1P2/6KP/8 b - - 0 37
Game processeda.
5k2/8/5p2/8/7p/5PqP/Q7/7K w - - 0 69
Game processeda.
5k2/8/5p2/8/7p/5PqP/Q7/7K w - - 0 69
Game processeda.
5k2/8/5p2/8/7p/5PqP/Q7/7K w - - 0 69
empty game
8/p7/1p1pQq2/1P1P2k1/2P1P1p1/3K4/8/8 w - - 0 61
Game processeda.
8/p7/1p1pQq2/1P1P2k1/2P1P1p1/3K4/8/8 w - - 0 61
Game processeda.
8/p7/1p1pQq2/1P1P2k1/2P1P1p1/3K4/8/8 w - - 0 61
Game processeda.
8/p6k/1p6/3q3P/1P2p1Q1/P7/6P1/6K1 w - - 0 32
Game processeda.
6k1/5pp1/1p6/1P2Q1P1/3P4/7q/5P2/6K1 w - - 0 42
Game processeda.
8/7Q/3pqkp1/2p5/2P5/6PP/P4P2/6K1 b - - 0 36
Game processeda.
8/7Q/3pqkp1/2p5/2P5/6PP/P4P2/6K1 b - - 0 36
Game processeda.
8/1p4k1/3p1p2/2P2P2/1P1QP1p1/8/7q/5K2 b - - 0 46
Game processeda.
8/8/8/3p4/5Q2/2P5/2K3k1/4q3 w - - 0 90
Game processeda.
8/8/8/3p4/5Q2/2P5/2K3k1/4q3 w - - 0 90
Game processeda.
8/5pk1/1p4p1/p1p4p

In [36]:
len(gamechap)

220

In [37]:
gameNo

219